In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from lifelines import*
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from lifelines.utils import median_survival_times
from lifelines import CoxPHFitter  #cox-proportional hazard ratio
import matplotlib.pyplot as plt
kmf=KaplanMeierFitter()
cph=CoxPHFitter()
#cph=CoxPHFitter(penalizer=0.1)

In [ ]:
df = pd.read_excel(LocalFolder + "/Data.xlsx")

In [ ]:
#Categorical (fixed cutoff, such as median cutoff, mean cutoffs)
def ClassbyOneThreshold(dTest, cutoff):
    cutoff=0.5
    cut = dTest.quantile([cutoff])
    catgr, gPt =[],[]
    for i in np.arange(3, len(dTest.columns)):
        g1=dTest[(dTest.iloc[:,i]>cut.iloc[0,i-1])]
        g2=dTest[(dTest.iloc[:,i]<=cut.iloc[0,i-1])]
        g1["Class%s"%dTest.columns[i]]=1
        g2["Class%s"%dTest.columns[i]]=0
        g = pd.concat([g1["Class%s"%dTest.columns[i]], g2["Class%s"%dTest.columns[i]]])
        gPt.append(pd.concat([g1,g2]))
        catgr.append(g)
        catgr_all=pd.DataFrame(catgr).transpose()
        complete_all=pd.concat(gPt,axis=1)
        dfClass0 = complete_all.loc[:,~complete_all.columns.duplicated()]
        dfClass = dfClass0.iloc[:, np.r_[0,1,2,len(features.columns)+3:len(dfClass0.columns)]]
    dfClass.to_csv(LocalFolder+"\os\MedianCutOffClassData.txt")
    return(dfClass)


# ROC curve for best cutoff
def RocForCutOff(dTest): 
    Results ={"Vars":[],"Data_cutoff":[], "ROC_AUC":[]}
    for i in np.arange(3, len(dTest.columns)):  
        label = dTest.iloc[:,2].values.astype(int)
        predictors = dTest.iloc[:,i].values.reshape(-1,1)
        model = LogisticRegression()
        model.fit(predictors, label)
        probs = model.predict_proba(predictors)

        fpr, tpr, thresholds = roc_curve(label, probs[:,1]) 
        roc_auc = auc(fpr, tpr)
        
        plt.figure()
        plt.plot(fpr, tpr, lw=1, label='ROC/AUC (area = %0.2f)' % (roc_auc))
        plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.grid()
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC %s'%dTest.columns[i])
        plt.legend(loc="lower right")
        plt.savefig(LocalFolder+"/os/Roc%s.tiff"% dTest.columns[i].split('/')[0], format='tiff',dpi=600)
        plt.show() # comment out for multiple lines plot together


        optimal_thresholds = thresholds[np.argmax(tpr-fpr)]  
        data = pd.DataFrame({"%s" %dTest.columns[i]:dTest.iloc[:,i].values, 
                            "Labels":label, "Probs":probs[:,1]}).sort_values(by=["Probs"])
        cutoff = data.loc[data[data.columns[2]] == optimal_thresholds][[data.columns[0]]].iloc[0,0]

        Results["Vars"].append(dTest.columns[i])
        Results["Data_cutoff"].append(cutoff)
        Results["ROC_AUC"].append(roc_auc)


    Res = pd.DataFrame(Results)
    Res.to_csv(LocalFolder+"\os\ROC_CutOffAUC.txt")
    return(Res)


# Create feature-dependent cutoffs based on ROC curve:
def ClassbyRoc(dTest, Res):
    catgr, gPt =[],[]
    for i in np.arange(3, len(dTest.columns)):
        g1=dTest[(dTest.iloc[:,i]>Res.iloc[:,1][i-3])]
        g2=dTest[(dTest.iloc[:,i]<=Res.iloc[:,1][i-3])]
        g1["Class%s"%dTest.columns[i]]=1
        g2["Class%s"%dTest.columns[i]]=0
        g = pd.concat([g1["Class%s"%dTest.columns[i]], g2["Class%s"%dTest.columns[i]]])
        gPt.append(pd.concat([g1,g2]))
        catgr.append(g)
        catgr_all=pd.DataFrame(catgr).transpose()
        complete_all=pd.concat(gPt,axis=1)
        dfClass0 = complete_all.loc[:,~complete_all.columns.duplicated()]
        dfClassbyROC = dfClass0.iloc[:, np.r_[0,1,2,len(features.columns)+3:len(dfClass0.columns)]]
    dfClassbyROC.to_csv(LocalFolder+"\os\ClassbyRocData.txt")


    FrequencyCount=[]
    for i in np.arange(3, len(dTest.columns)):
        FrequencyCount.append(dfClassbyROC["Class%s"%dTest.columns[i]].value_counts())
        Freq=pd.DataFrame(FrequencyCount)
    Freq["From Lower cutoff"]=100*Freq[0]/(Freq[0]+Freq[1])
    Freq.to_csv(LocalFolder+"\os\RocThresholdCutoffs.txt")
    return(dfClassbyROC)


      


# ROC for all predictors (can also put multiple, most powerful ones)
def RocAllPredictor(dTest):
    predictors = dTest.iloc[:,np.r_[3:len(dTest.columns)]].values
    label = dTest.iloc[:,2].values.astype(int)

    model = LogisticRegression()
    model.fit(predictors, label)
    probs = model.predict_proba(predictors)

    fpr, tpr, thresholds = roc_curve(label, probs[:,1]) 
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='ROC/AUC (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.grid()

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC_all predictors')
    plt.legend(loc="lower right")
    plt.savefig(LocalFolder+"/os/allROCAUC.pdf")
    plt.show()
    


# Cox Hazard Ratio- Mutivariate
def Cox(df):
    df = df.drop(columns=df.columns[0])
    c=cph.fit(df, duration_col=df.columns[0], event_col=df.columns[1])
    bx=c.plot(hazard_ratios=True)
    

# Cox Hazard Ratio- CI of every feature 
def CoxIndvidualFeature(dTest):
    for i in np.arange(3, len(dTest.columns)): 
        columnOfInterest = dTest.columns[i]
        dataOfInterest = dTest[[dTest.columns[1],dTest.columns[2],columnOfInterest]]
        c=cph.fit(dataOfInterest, duration_col=dTest.columns[1], event_col=dTest.columns[2])
        c.print_summary()
        plt.figure()
        bx=c.plot(hazard_ratios=True)
        bx.get_figure().tight_layout()
        bx.get_figure().savefig(LocalFolder+"/os/%s.tiff"% dTest.columns[i].split('/')[0], format='tiff',dpi=600)
        plt.show()


# KM plot for every feature
def KMplot(dTest):
    for i in np.arange(3, len(dTest.columns)):
        g_high = dTest[(dTest.iloc[:,i]==1)]
        g_low = dTest[(dTest.iloc[:,i]==0)]

        plt.figure()
        print("KM curves")
        kmf1=kmf.fit(g_high[dTest.columns[1]], g_high[dTest.columns[2]], label="Hi %s"%dTest.columns[i])
        ax=kmf.plot(color="red", ci_show=False,show_censors=True)

        kmf1=kmf.fit(g_low[dTest.columns[1]], g_low[dTest.columns[2]], label="Low")
        ax=kmf.plot(color="black", ci_show=False,show_censors=True)
        plt.ylim(0,1.1)

        ax.get_figure().savefig(LocalFolder+"\os\KM%s.png"%dTest.columns[i].split('/')[0], dpi=600)
        

# Logrank for every features (once calssified):
def logrank(dTest):
    Logp, Vars  =[],[]
    for i in np.arange(3, len(dTest.columns)):
        g_high = dTest[(dTest.iloc[:,i]==1)]
        g_low = dTest[(dTest.iloc[:,i]==0)]
        lr = logrank_test(g_high[dTest.columns[1]], g_low[dTest.columns[1]], g_high[dTest.columns[2]], g_low[dTest.columns[2]])
        Vars.append(dTest.columns[i])
        Logp.append(lr.p_value)
    Log=pd.DataFrame({"logp":Logp, "Vars":Vars})
    Log.to_csv(LocalFolder+"\os\Logrankp.txt")
    print("success")
    Log

    
def main():
    # Use ClassbyRoc
    #dTest=dataPfs
    #ClassbyRoc(dTest, RocForCutOff(dTest))
    #RocAllPredictor(ClassbyRoc(dTest, RocForCutOff(dTest)))
    #Cox(ClassbyRoc(dTest, RocForCutOff(dTest)))
    #CoxIndvidualFeature(ClassbyRoc(dTest, RocForCutOff(dTest)))
    #KMplot(ClassbyRoc(dTest, RocForCutOff(dTest)))
    #logrank(ClassbyRoc(dTest, RocForCutOff(dTest)))
    
    # Use ClassbyMedian or percentile 
    dTest=dataPfs
    cutoff = 0.5
    RocAllPredictor(ClassbyOneThreshold(dTest, cutoff))
    Cox(ClassbyOneThreshold(dTest, cutoff))
    CoxIndvidualFeature(ClassbyOneThreshold(dTest, cutoff))
    KMplot(ClassbyOneThreshold(dTest, cutoff))
    logrank(ClassbyOneThreshold(dTest, cutoff))
    

if __name__ == '__main__':
    main()

    